In [1]:
# TODO: Remove this before submission
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


In [2]:
import pandas as pd
import numpy as np
import nltk

nltk.download("wordnet")
import re
from bs4 import BeautifulSoup


[nltk_data] Downloading package wordnet to /Users/aditya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz

## Read Data


## Keep Reviews and Ratings


In [4]:
# Load the tab separated data file, and print the first 5 rows for confirmation
data = pd.read_csv("./data/amazon_reviews_us_Jewelry_v1_00.tsv", sep="\t", usecols=["star_rating", "review_body"])
data.head()


/var/folders/d7/gcxbjppx70qfkdwy32kg04dh0000gn/T/ipykernel_37527/949400013.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("./data/amazon_reviews_us_Jewelry_v1_00.tsv", sep="\t", usecols=["star_rating", "review_body"])


,star_rating,review_body
0,5,so beautiful even tho clearly not high end ......
1,5,"Great product.. I got this set for my mother, ..."
2,5,Exactly as pictured and my daughter's friend l...
3,5,Love it. Fits great. Super comfortable and nea...
4,5,Got this as a Mother's Day gift for my Mom and...


Understanding Data


In [5]:
data.describe()


,star_rating,review_body
count,1767042,1766807
unique,11,1618522
top,5,Love it
freq,1041056,4288


In [6]:
data.star_rating.unique()


array([5, 1, 4, 3, 2, nan, '5', '1', '3', '4', '2', '2012-12-21'],
      dtype=object)

In [7]:
data.groupby(["star_rating"]).count()


,review_body
star_rating,
1,150441
2,97259
3,153660
4,259019
5,1040896
1,4566
2,3541
2012-12-21,0
3,5999


In [8]:
# Drop the outlier which is star_rating = "2012-12-21"
data = data[data.star_rating != "2012-12-21"]


In [9]:
# Remove nan valued rows
data = data[data.star_rating.notnull()]


In [10]:
data.describe()


,star_rating,review_body
count,1767041,1766807
unique,10,1618522
top,5,Love it
freq,1041056,4288


In [11]:
# Convert all star rating to integer
data["star_rating"] = data.star_rating.astype(int)


In [12]:
data.head()


,star_rating,review_body
0,5,so beautiful even tho clearly not high end ......
1,5,"Great product.. I got this set for my mother, ..."
2,5,Exactly as pictured and my daughter's friend l...
3,5,Love it. Fits great. Super comfortable and nea...
4,5,Got this as a Mother's Day gift for my Mom and...


In [13]:
data = data[data.review_body.notnull()]
data.head()


,star_rating,review_body
0,5,so beautiful even tho clearly not high end ......
1,5,"Great product.. I got this set for my mother, ..."
2,5,Exactly as pictured and my daughter's friend l...
3,5,Love it. Fits great. Super comfortable and nea...
4,5,Got this as a Mother's Day gift for my Mom and...


In [14]:
# There are no empty reviews
(data.review_body.str.len() <= 0).sum()


0

Now we can continue with the process.


## We select 20000 reviews randomly from each rating class.


In [15]:
# np.random.seed(101)
N_SAMPLES = 25000


In [16]:
sampled_data = data.groupby("star_rating", group_keys=False).apply(lambda x: x.sample(N_SAMPLES))


In [17]:
# TODO: Remove Test Block
# Checkout: Sampled Review, Ratings Data
# Save data to a file for having a look at the text content

sampled_data.head()
sampled_data.to_csv("./data/sampled.tsv", sep="\t", index=False, encoding="UTF-8")


,star_rating,review_body
826126,1,Is too tight I don't like! Could i return it?
1629377,1,These earrings were beautiful and just the rig...
721257,1,Baaaaaad
330874,1,Looks cheap.
1209952,1,While this bracelet is very pretty it is so sm...


# Data Cleaning


In [18]:
avg_len_before_cleaning = sampled_data.review_body.str.len().mean()
f"Avg. length of reviews BEFORE CLEANING :: {avg_len_before_cleaning}"


'Avg. length of reviews BEFORE CLEANING :: 189.236704'

In [19]:
# TODO: Remove Test Block
# Test to lower case
sampled_data["review_body"].str.lower()


826126         is too tight i don't like! could i return it?
1629377    these earrings were beautiful and just the rig...
721257                                              baaaaaad
330874                                          looks cheap.
1209952    while this bracelet is very pretty it is so sm...
                                 ...                        
160088                               love these. nicely made
714338                                           so cute thx
710107     make your special lady for like a princess, ni...
1179584    very detailed, more than they look in the phot...
1634249    this bead is of high quality. it was priced lo...
Name: review_body, Length: 125000, dtype: object

In [20]:
# Convert all reviews to lower case (optional according to study)
def to_lower(data: pd.Series):
    return data.str.lower()


In [21]:
# TODO: Remove Test Block
import unicodedata

# Remove accented characters
print(
    unicodedata.normalize(
        "NFKD", "Sómě Áccěntěd těxt. Some words such as résumé, café, prótest, divorcé, coördinate, exposé, latté."
    )
    .encode("ascii", "ignore")
    .decode("utf-8", "ignore")
)


Some Accented text. Some words such as resume, cafe, protest, divorce, coordinate, expose, latte.


In [22]:
def remove_accented_characters(data: pd.Series):
    import unicodedata

    """Removes accented characters from the Series

    Args:
        data (pd.Series): Series of string

    Returns:
        _type_: pd.Series
    """
    import unicodedata

    return data.apply(lambda x: unicodedata.normalize("NFKD", x).encode("ascii", "ignore").decode("utf-8", "ignore"))


In [23]:
# TODO: Remove Test Block
# Test HTML and URL replace RegEx
import re

# Removing HTML tags with and without closing tags
re.sub(
    r"<[a-zA-Z]+\s?/?>",
    " ",
    "this is a string <containing> some random XML tags <br> which might not be closed like this <br /> or this <br/>",
)

# Removing HTML encoded characters.
re.sub(r"&#\d+;", " ", "why are &#34; you doing this?")

# Removing URLs
re.sub(
    r"https?://([\w\-\._]+){2,}/[\w\-\.\-/=\+_\?]+",
    " ",
    "Ruby Garnet Ring<br />I saw this ring after I had already purchased 4 rings for my birthday.  I just could not resist buying this ring because of its color (and the price was so low I couldn't believe it!)<br />It arrived two days before my birthday in a small black velvet ring bag with a draw string.  It was so pretty and so perfect.  It looked like Ruby during the daylight.  But at night, under incandescent lighting, it looked garnet.  I have owned and worn many, many rings in the past 10 years; but this ring looked the best of all on my hand.  I have decided to name this ring:<br />Ruby Garnet for a Renewed America<br />And I wrote a little poem for meditation to go along with it:<br /><br />Ruby Garnet for a Renewed America<br />Ruby by day:<br />Giving freely each day<br />Is to give `the Juliet way':<br /><br />\\"
    "The more I give to you, the more I have;<br />For both are infinite.\\"
    "<br /><br />To give freely is right;<br />Standing favorably in Our Saviour's sight.<br /><br />Garnet by night:<br />To stand right in Our Saviour's sight;<br />This is our health;<br />And all of our wealth. --- Gwen Newclear The Poet<br /><br />I thought I lost this ring:  found it.  http://poetgwennewclear - dot - sharepoint - dot - com https://www.gogole.com/asd___asd?test=1",
)


'this is a string   some random XML tags   which might not be closed like this   or this  '

'why are   you doing this?'

"Ruby Garnet Ring<br />I saw this ring after I had already purchased 4 rings for my birthday.  I just could not resist buying this ring because of its color (and the price was so low I couldn't believe it!)<br />It arrived two days before my birthday in a small black velvet ring bag with a draw string.  It was so pretty and so perfect.  It looked like Ruby during the daylight.  But at night, under incandescent lighting, it looked garnet.  I have owned and worn many, many rings in the past 10 years; but this ring looked the best of all on my hand.  I have decided to name this ring:<br />Ruby Garnet for a Renewed America<br />And I wrote a little poem for meditation to go along with it:<br /><br />Ruby Garnet for a Renewed America<br />Ruby by day:<br />Giving freely each day<br />Is to give `the Juliet way':<br /><br />\\The more I give to you, the more I have;<br />For both are infinite.\\<br /><br />To give freely is right;<br />Standing favorably in Our Saviour's sight.<br /><br />Ga

In [24]:
def remove_html_and_url(data: pd.Series):
    """Function to remove
             1. HTML encodings
             2. HTML tags (both closed and open)
             3. URLs

    Args:
        data (pd.Series): A Pandas series of type string

    Returns:
        _type_: pd.Series
    """
    # Remove HTML encodings
    data.str.replace(r"&#\d+;", " ", regex=True)

    # Remove HTML tags (both open and closed)
    data.str.replace(r"<[a-zA-Z]+\s?/?>", " ", regex=True)

    # Remove URLs
    data.str.replace(r"https?://([\w\-\._]+){2,}/[\w\-\.\-/=\+_\?]+", " ", regex=True)

    return data


In [25]:
# TODO: Remove Test Block
# Test Non-alpha characters replace RegEX
import re

# Remove non-alphabetical characters
re.sub(r"\\|[^a-zA-z0-9\s]", " ", "33245 sure# if this % was #not! 55888392923 What do# you will** of it.? $$^!")
re.sub(
    r"\\|[^a-zA-z\s]",
    " ",
    "receive necklace gift disappointed imperfection \\pearls\\ look close enoughthe coating arrive black box inside drawstring bag inside plastic ziploc bag \\made china\\ tagsending back",
)


'33245 sure  if this   was  not  55888392923 What do  you will   of it     ^ '

'receive necklace gift disappointed imperfection  pearls  look close enoughthe coating arrive black box inside drawstring bag inside plastic ziploc bag  made china  tagsending back'

In [26]:
# Remove non-alphabetical characters
def remove_non_alpa_characters(data: pd.Series):
    return data.str.replace(r"_+|\\|[^a-zA-z0-9\s]", " ", regex=True)


In [27]:
# TODO: Remove Test Block
# Test remove extra space RegEX
import re

re.sub(r"^\s*|\s\s*", " ", "sameple of the    template, where nothing \t is defined.")


' sameple of the template, where nothing is defined.'

In [28]:
# Remove extra spaces
def remove_extra_spaces(data: pd.Series):
    return data.str.replace(r"^\s*|\s\s*", " ", regex=True)


In [29]:
# Install contractions package, if you don't have it
! pip install contractions

In [30]:
# TODO: Remove Test Block
# Test expanding of contractions
import contractions

sample = "I was disappointed with this purchase, they were extremely small, can't even wear them, they should specify the size, I would've never purchased them if I had known... Thanks"
expanded = " ".join([contractions.fix(word) for word in sample.split()])
expanded


'I was disappointed with this purchase, they were extremely small, cannot even wear them, they should specify the size, I would have never purchased them if I had known... Thanks'

In [31]:
# Expanding contractions
def fix_contractions(data: pd.Series):
    import contractions

    def contraction_fixer(txt: str):
        return " ".join([contractions.fix(word) for word in txt.split()])

    return data.apply(contraction_fixer)


In [32]:
# A dictionary containing the columns and a list of functions to perform on it in order
# TODO: Think about handling negation.
# TODO: Replace numbers with NUM tag.
data_cleaning_pipeline = {
    "review_body": [
        to_lower,
        remove_accented_characters,
        remove_html_and_url,
        fix_contractions,
        remove_non_alpa_characters,
        remove_extra_spaces,
    ]
}

cleaned_data = sampled_data.copy()

# Process all the cleaning instructions
for col, pipeline in data_cleaning_pipeline.items():
    # Get the column to perform cleaning on
    temp_data = data[col]

    # Perform all the cleaning functions sequencially
    for func in pipeline:
        print(f"Starting: {func.__name__}")
        temp_data = func(temp_data)
        print(f"Ended: {func.__name__}")

    # Replace the old column with cleaned one.
    cleaned_data[col] = temp_data


Starting: to_lower
Ended: to_lower
Starting: remove_accented_characters
Ended: remove_accented_characters
Starting: remove_html_and_url
Ended: remove_html_and_url
Starting: fix_contractions
Ended: fix_contractions
Starting: remove_non_alpa_characters
Ended: remove_non_alpa_characters
Starting: remove_extra_spaces
Ended: remove_extra_spaces


In [33]:
avg_len_after_cleaning = cleaned_data.review_body.str.len().mean()
f"Avg. length of reviews after cleaning :: {avg_len_after_cleaning}"


'Avg. length of reviews after cleaning :: 185.766312'

In [34]:
# TODO: Remove Test Block
# Checkpoint: Cleaned Data
cleaned_data.to_csv("./data/cleaned.tsv", sep="\t", index=False, encoding="UTF-8")


# Pre-processing


## remove the stop words


In [68]:
# TODO: Remove Test Block
cleaned_data = pd.read_csv("./data/cleaned.tsv", sep="\t")


In [69]:
avg_len_before_preprocessing = cleaned_data["review_body"].str.len().mean()
print(f"Avg. length of the reviews before preprocessing :: {avg_len_before_preprocessing}")


Avg. length of the reviews before preprocessing :: 185.127688


In [70]:
def tokenize(data: pd.Series):
    from nltk.tokenize import word_tokenize

    nltk.download("punkt")

    return data.apply(word_tokenize)


In [71]:
# 1. Tokenize the words
tokenized_data = cleaned_data.copy()
tokenized_data["review_body"] = tokenize(cleaned_data.review_body)
tokenized_data.head()


[nltk_data] Downloading package punkt to /Users/aditya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,star_rating,review_body
0,1,"[it, arrived, with, the, locket, broken, and, ..."
1,1,"[ordered, as, last, minute, christmas, gift, f..."
2,1,"[it, gave, me, the, opposite, all, i, got, fr,..."
3,1,"[shity, belly, button, rings, and, eyebrow, no..."
4,1,"[the, bead, is, cute, but, very, small, it, do..."


In [72]:
import pickle as pkl

with open("./data/tokenized_data.pkl", "wb") as file:
    pkl.dump(tokenized_data, file)


In [73]:
from typing import List, Set


def remove_stopwords(data: pd.Series):
    """Remove stop words using the NLTK stopwords dictionary

    Args:
        string (str): a document

    Returns:
        str: a document with stopwords removed
    """
    from nltk.corpus import stopwords

    nltk.download("stopwords")

    stopwords = set(stopwords.words())

    def remover(word_list: List[str], stopwords: Set[str]):
        return [word for word in word_list if not word in stopwords]

    return data.apply(lambda word_list: remover(word_list, stopwords))


In [74]:
removed_stopwords = tokenized_data.copy()
removed_stopwords["review_body"] = remove_stopwords(tokenized_data["review_body"])
removed_stopwords.head()


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aditya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,star_rating,review_body
0,1,"[arrived, locket, broken, chain, different, pi..."
1,1,"[ordered, minute, christmas, gift, yr, girl, f..."
2,1,"[gave, opposite, fr, negative, energy, feel, w..."
3,1,"[shity, belly, button, rings, eyebrow, labret,..."
4,1,"[bead, cute, small, fit, silver, pandora, brac..."


In [75]:
import pickle as pkl

with open("./data/removed_stopwords.pkl", "wb") as file:
    pkl.dump(removed_stopwords, file)


## perform lemmatization


In [76]:
import pickle as pkl

removed_stopwords = None
with open("./data/removed_stopwords.pkl", "rb") as file:
    removed_stopwords = pkl.load(file)


In [79]:
def perform_wordnet_lemmatization(data: pd.Series, consider_pos_tag: bool = True):
    from nltk.corpus import wordnet
    from nltk.stem import WordNetLemmatizer

    nltk.download("omw-1.4")

    # 1. Perform POS tagging
    def perform_nltk_pos_tag(data: pd.Series):
        from nltk import pos_tag

        nltk.download("averaged_perceptron_tagger")

        return data.apply(pos_tag)

    def wordnet_pos_tagger(tag: str):
        if tag.startswith("J"):
            return wordnet.ADJ
        elif tag.startswith("V"):
            return wordnet.VERB
        elif tag.startswith("N"):
            return wordnet.NOUN
        elif tag.startswith("R"):
            return wordnet.ADV
        else:
            return None

    if consider_pos_tag:
        pos_tagged_data = data.copy()
        pos_tagged_data["review_body"] = perform_nltk_pos_tag(data["review_body"])

    lemmatizer = WordNetLemmatizer()
    lemmatized = list()

    for row in data:

        lemmatized_row = list()

        for word, tag in row:
            if consider_pos_tag:
                wordnet_pos_tag = wordnet_pos_tagger(tag)

                if wordnet_pos_tag is None:
                    lemmatized_row.append(word)
                else:
                    result = lemmatizer.lemmatize(word, wordnet_pos_tag)
                    lemmatized_row.append(lemmatizer.lemmatize(word, wordnet_pos_tag))
            else:
                lemmatized_row.append(word)
                
        lemmatized.append(lemmatized_row)

    return lemmatized


In [80]:
lemmatized_data = removed_stopwords.copy()
lemmatized_data["review_body"] = perform_wordnet_lemmatization(removed_stopwords["review_body"], consider_pos_tag=False)
lemmatized_data.head()


[nltk_data] Downloading package omw-1.4 to /Users/aditya/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,star_rating,review_body
0,1,"[arrive, locket, broken, chain, different, pie..."
1,1,"[order, minute, christmas, gift, yr, girl, flo..."
2,1,"[give, opposite, fr, negative, energy, feel, w..."
3,1,"[shity, belly, button, ring, eyebrow, labret, ..."
4,1,"[bead, cute, small, fit, silver, pandora, brac..."


In [81]:
# TODO: Remove Test Block
# Save lemmatized data
import pickle as pkl

with open("./data/lemamtized_data.pkl", "wb") as file:
    pkl.dump(lemmatized_data, file)


In [82]:
avg_len_after_preprocessing = lemmatized_data["review_body"].str.len().mean()
print(f"Avg. length of the reviews after preprocessing :: {avg_len_after_preprocessing}")


Avg. length of the reviews after preprocessing :: 12.766992


# TF-IDF Feature Extraction


In [83]:
# TODO: Remove Test Block
# Load lemmatized data
import pickle as pkl

lemmatized_data = None
with open("./data/lemamtized_data.pkl", "rb") as file:
    lemmatized_data = pkl.load(file)


In [84]:
sample = lemmatized_data.loc[
    :5,
]
sample["review_body"].apply(lambda words: " ".join(words))


0    arrive locket broken chain different piece kin...
1    order minute christmas gift yr girl flower pin...
2          give opposite fr negative energy feel waste
3    shity belly button ring eyebrow labret metal h...
4    bead cute small fit silver pandora bracelet le...
5    open package remove item incredulous necklace ...
Name: review_body, dtype: object

In [85]:
# Concatenate lemmatized sentences back into one sentence
def concatenator(data: pd.Series):
    return data.apply(lambda words: " ".join(words))


In [86]:
final_data = lemmatized_data.copy()
final_data["review_body"] = concatenator(lemmatized_data["review_body"])
final_data.head()


,star_rating,review_body
0,1,arrive locket broken chain different piece kin...
1,1,order minute christmas gift yr girl flower pin...
2,1,give opposite fr negative energy feel waste
3,1,shity belly button ring eyebrow labret metal h...
4,1,bead cute small fit silver pandora bracelet le...


In [98]:
final_data[final_data["review_body"].str.len() == 0].groupby("star_rating").count()


,review_body
star_rating,


In [88]:
final_data["review_body"].isnull().values.any(), final_data["review_body"].isnull().sum()


(False, 0)

In [89]:
# Drop empty strings
final_data["review_body"].replace("", np.nan, inplace=True)
final_data.dropna(subset=["review_body"], inplace=True)
final_data[final_data["review_body"].str.len() == 0].groupby("star_rating").count()
final_data["review_body"].isnull().values.any(), final_data["review_body"].isnull().sum()


,review_body
star_rating,


(False, 0)

In [90]:
final_data["review_body"].isna().sum()


0

In [99]:
final_data = final_data.groupby("star_rating", group_keys=False).apply(lambda x: x.sample(20000))


In [100]:
# TODO: Remove Checkpoint ...
final_data.to_csv("./data/data.tsv", sep="\t", index=False)


In [101]:
# TODO: Remove this block
# Retrieve checkpoint
data = pd.read_csv("./data/data.tsv", sep="\t")


In [102]:
# Split the data 80-20 split
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, stratify=data["star_rating"])


In [103]:
(train["review_body"].str.len() == 0).sum()


0

In [104]:
data.groupby(["star_rating"]).count()


,review_body
star_rating,
1,20000
2,20000
3,20000
4,20000
5,20000


In [105]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(train["review_body"])

X_tfidf_train = vectorizer.transform(train["review_body"])
X_tfidf_test = vectorizer.transform(test["review_body"])
y_train = train["star_rating"]
y_test = test["star_rating"]


TfidfVectorizer()

TODO: Use cross validation. KFold, StratifiedKFold


### Helper Functions

In [106]:
def calc_metrics(y_true, y_pred):
    from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

    accuracy_score = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)
    f1 = f1_score(y_true, y_pred, average=None)

    return {
        "accuracy": accuracy_score,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "avg.": {"precision": np.mean(precision), "recall": np.mean(recall), "f1": np.mean(f1)},
    }


# Perceptron


In [107]:
from sklearn.linear_model import Perceptron

clf = Perceptron()
clf.fit(X_tfidf_train, y_train)

y_pred = clf.predict(X_tfidf_test)

calc_metrics(y_test, y_pred)


Perceptron()

{'accuracy': 0.3657,
 'precision': array([0.46713287, 0.32766497, 0.27074148, 0.32341311, 0.53828996]),
 'recall': array([0.4175 , 0.32275, 0.33775, 0.3885 , 0.362  ]),
 'f1': array([0.44092409, 0.32518892, 0.30055617, 0.35298126, 0.4328849 ]),
 'avg.': {'precision': 0.38544847977714375,
  'recall': 0.3657,
  'f1': 0.370507069259875}}

# SVM


In [108]:
from sklearn.svm import LinearSVC

clf = LinearSVC(dual=False)
clf.fit(X_tfidf_train, y_train)

y_pred = clf.predict(X_tfidf_test)

calc_metrics(y_test, y_pred)


# Logistic Regression


In [25]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

skf = StratifiedKFold(n_splits=10)
n_fold = 1

clf = LogisticRegression(solver="lbfgs", multi_class="multinomial")

for train_index, test_index in skf.split(data, data['star_rating']):
  train = data.loc[train_index,:]
  test = data.loc[test_index,:]

  vectorizer = TfidfVectorizer()
  vectorizer.fit(train["review_body"])

  X_tfidf_train = vectorizer.transform(train["review_body"])
  X_tfidf_test = vectorizer.transform(test["review_body"])
  y_train = train["star_rating"]
  y_test = test["star_rating"]

  clf.fit(X_tfidf_train, y_train)

  y_pred = clf.predict(X_tfidf_test)

  print(f"Fold:: {n_fold}")
  calc_metrics(y_test, y_pred)
  n_fold += 1

TfidfVectorizer()

/Users/aditya/.pyenv/versions/3.10.6/envs/csci544-assignment-01/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

Fold:: 1


{'accuracy': 0.4771,
 'precision': array([0.56838021, 0.38745981, 0.38630685, 0.41740088, 0.59645233]),
 'recall': array([0.586 , 0.3615, 0.3865, 0.379 , 0.6725]),
 'f1': array([0.57705564, 0.37403001, 0.3864034 , 0.39727463, 0.63219741]),
 'avg.': {'precision': 0.47120001525052874,
  'recall': 0.47709999999999997,
  'f1': 0.4733922179740512}}

...

TfidfVectorizer()

/Users/aditya/.pyenv/versions/3.10.6/envs/csci544-assignment-01/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

Fold:: 10


{'accuracy': 0.4737,
 'precision': array([0.54637282, 0.3870595 , 0.37038835, 0.42208857, 0.60944596]),
 'recall': array([0.595 , 0.335 , 0.3815, 0.386 , 0.671 ]),
 'f1': array([0.56965055, 0.35915304, 0.37586207, 0.40323844, 0.63874346]),
 'avg.': {'precision': 0.4670710406005029,
  'recall': 0.4737,
  'f1': 0.4693295121006451}}

In [20]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver="saga", max_iter=400, multi_class="multinomial")
clf.fit(X_tfidf_train, y_train)

y_pred = clf.predict(X_tfidf_test)

calc_metrics(y_test, y_pred)


LogisticRegression(max_iter=400, multi_class='multinomial', solver='saga')

{'precision': array([0.56120827, 0.38899587, 0.38057609, 0.4338255 , 0.6216968 ]),
 'recall': array([0.61775, 0.3535 , 0.37325, 0.404  , 0.6705 ]),
 'f1': array([0.58812329, 0.37039948, 0.37687745, 0.41838188, 0.64517681]),
 'avg.': {'precision': 0.4772605069477551,
  'recall': 0.4838,
  'f1': 0.47979177960655406}}

# Naive Bayes


In [14]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_tfidf_train, y_train)

y_pred = clf.predict(X_tfidf_test)

calc_metrics(y_test, y_pred)


MultinomialNB()

{'precision': array([0.57384276, 0.3728278 , 0.36901271, 0.42162753, 0.61076747]),
 'recall': array([0.58575, 0.354  , 0.3775 , 0.386  , 0.6665 ]),
 'f1': array([0.57973525, 0.36317004, 0.37320811, 0.40302793, 0.63741781]),
 'avg.': {'precision': 0.4696156537891699,
  'recall': 0.47395000000000004,
  'f1': 0.471311827908986}}